In [11]:
import torch
import torch.nn.functional as F
import math
from HTorch import HParameter, HTensor

In [2]:
class HEmbedding(torch.nn.Embedding):
    def __init__(self, *args, manifold='PoincareBall', curvature=-1.0, **kwargs):
        super(HEmbedding, self).__init__(*args, **kwargs)
        self.weight = HParameter(self.weight, manifold=manifold, curvature=curvature)
        self.weight.init_weights()
    
    def forward(self, input):
        output = super().forward(input).as_subclass(HTensor)
        output.manifold = self.weight.manifold
        output.curvature = self.weight.curvature
        return output

In [3]:
curv = -1.2
curv = torch.nn.Parameter(torch.tensor(curv))

In [20]:
x = HTensor([2,2], curvature = curv).proj_()
print(x.curvature.is_leaf)

True


In [4]:
a = HEmbedding(3,2,curvature=curv)
x = torch.tensor([0,1]).long()
emb = a(x)
print(emb)
print(list(a.parameters()))

Hyperbolic tensor([[ 9.6396e-06,  1.4593e-06],
        [-8.5606e-06, -1.1475e-06]], grad_fn=<AliasBackward0>), manifold=PoincareBall, curvature=-1.2000000476837158
[Hyperbolic Parameter containing:
tensor([[ 9.6396e-06,  1.4593e-06],
        [-8.5606e-06, -1.1475e-06],
        [-8.6799e-06, -5.0161e-06]], requires_grad=True), manifold=PoincareBall, curvature=-1.2000000476837158]


In [5]:
class HypLinear(torch.nn.Linear):
    """
    Hyperbolic linear layer.
    """
    def __init__(self, *args, dropout=0.0, **kwargs):
        super(HypLinear, self).__init__(*args, **kwargs)
        self.reset_parameters()
        self.dropout = dropout
        
    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight, gain=math.sqrt(2))
        torch.nn.init.constant_(self.bias, 0)

    def forward(self, x:HTensor):
        drop_weight = F.dropout(self.weight, self.dropout, training=self.training)
        res = x.mobius_matvec(drop_weight, x).proj()
        if self.bias is not None:
            bias = x.proj_tan0(self.bias.view(1, -1))
            hyp_bias = x.expmap0(bias).proj()
            res = res.mobius_add(res, hyp_bias).proj()
        return res
    
class HypAgg(torch.nn.Module):
    """
    Hyperbolic aggregation layer.
    """
    def __init__(self, in_features, dropout, use_att, local_agg):
        super(HypAgg, self).__init__()
        self.in_features = in_features
        self.dropout = dropout
        self.local_agg = local_agg
        self.use_att = use_att
        if self.use_att:
            self.att = DenseAtt(in_features, dropout)

    def forward(self, x: HTensor, adj):
        abs_curvature = abs(x.curvature)
        x_tangent = x.logmap0(x)
        if self.use_att:
            if self.local_agg:
                x_local_tangent = []
                for i in range(x.size(0)):
                    x_local_tangent.append(
                        x.logmap(x[i], x)
                    )
                x_local_tangent = torch.stack(x_local_tangent, dim=0)
                adj_att = self.att(x_tangent, adj)
                att_rep = adj_att.unsqueeze(-1) * x_local_tangent
                support_t = torch.sum(
                    adj_att.unsqueeze(-1) * x_local_tangent, dim=1)
                output = x.expmap(x, support_t).proj()
                return output
            else:
                adj_att = self.att(x_tangent, adj)
                support_t = torch.matmul(adj_att, x_tangent)
        else:
            support_t = torch.spmm(adj, x_tangent)
        output = x.expmap0(x, support_t).proj()
        return output
    
class HypAct(torch.nn.Module):
    """
    Hyperbolic activation layer.
    """
    def __init__(self, act, c_in, c_out):
        super(HypAct, self).__init__()
        # maybe we can give a manifold argument, and let it work across HTensors on different manifolds
        self.act = act
        self.c_in = c_in
        self.c_out = c_out

    def forward(self, x: HTensor):
        ## now can only work in one model
        assert self.c_in == x.curvature, "inconsistence curvatures between input and specified c_in"
        xt = self.act(x.logmap0(x))
        xt = x.manifold.proj_tan0(xt, c=self.c_out)
        res = x.manifold.expmap0(xt, c=self.c_out).as_subclass(HTensor)
        res.manifold = x.manifold
        res.curvature = self.c_out
        return res.proj()
    
    def extra_repr(self):
        return 'c_in={}, c_out={}'.format(
            self.c_in, self.c_out
        )

In [6]:
model = HypLinear(2,3)
print(model)

HypLinear(in_features=2, out_features=3, bias=True)


In [7]:
model.weight

Parameter containing:
tensor([[-0.7453, -0.3380],
        [ 0.0555, -1.1049],
        [ 1.2846,  1.4632]], requires_grad=True)

In [9]:
model(emb)

Hyperbolic tensor([[-7.6773e-06, -1.0775e-06,  1.4518e-05],
        [ 6.7678e-06,  7.9281e-07, -1.2676e-05]], grad_fn=<AliasBackward0>), manifold=PoincareBall, curvature=-1.2000000476837158